NETTOYAGE / FILTRAGE

In [1]:
import pandas as pd

In [ ]:
# Name Basics

'''
Colonnes à supprimer : Aucune
Lignes à supprimer:
    - knownfortitle = /N
    - primaryprofession = /N
'''

In [ ]:
# Title Akas

'''
Colonnes à supprimer : 
Lignes à supprimer: 
    - Faire un groupby par ID de film
    - A voir plus tard pour filtrer avec un contains sur language et/ou région
'''

In [ ]:
# Title Basics

'''
Colonnes à supprimer : 
Lignes à supprimer: 
    - Titletype = movie
    - A voir pour filtrer par années par la suite
    - A voir pour filtrer selon durée des films (trop courts ou trop longs)
    - isAdult = 1 et genre = adult (uniquement)
    - Vérifier classification de Basic instinct

'''

In [ ]:
# Title Crew

'''
RAS
'''

In [ ]:
# Title Episodes

'''
A voir : concatener numberEpisode et numberSeason
'''

In [ ]:
# Title Principals

'''
Colonnes à supprimer : job, character
'''

In [ ]:
# Title Ratings

'''
RAS
'''

In [ ]:
link = "./BD/tmdb_full.csv"
df_tmdb_full = pd.read_csv(link)

df_tmdb_full['fr?'] = df_tmdb_full['spoken_languages'].apply(lambda x: 'fr' in x)
df_fr = df_tmdb_full[df_tmdb_full['fr?']]
link = "./BD/title.akas.tsv.gz"
df_title_akas = pd.read_csv(link, compression = 'gzip', sep = '\t')
df_title_akas = df_title_akas[(df_title_akas['region'] == 'FR') | (df_title_akas['language'] == 'fr')]
df_TMDB_AKAS = pd.merge(df_fr,
                        df_title_akas,
                        how = 'left',
                        left_on = df_fr['imdb_id'],
                        right_on = df_title_akas['titleId'])

df_title_basics = pd.read_csv("BD/title.basics.tsv.gz", compression = 'gzip', sep = '\t')
df_TMDB_AKAS = df_TMDB_AKAS.drop(['key_0'], axis = 1)
df = pd.merge(df_TMDB_AKAS,
              df_title_basics,
              how = 'left',
              left_on = df_TMDB_AKAS['imdb_id'],
              right_on = df_title_basics['tconst'])
df_title_crew = pd.read_csv("BD/title.crew.tsv.gz", compression = 'gzip', sep = '\t')
df = df.drop(['key_0'], axis = 1)
df = pd.merge(df,
              df_title_crew,
              how = 'left',
              left_on = df['imdb_id'],
              right_on = df_title_crew['tconst'])
df_rate = pd.read_csv("BD/title.ratings.tsv.gz", compression = 'gzip', sep = '\t')
df = df.drop(['key_0'], axis = 1)
df = pd.merge(df,
              df_rate,
              how = 'left',
              left_on = df['imdb_id'],
              right_on = df_rate['tconst'])

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29164 entries, 0 to 29163
Data columns (total 50 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   key_0                         29164 non-null  object 
 1   adult                         29164 non-null  bool   
 2   backdrop_path                 19416 non-null  object 
 3   budget                        29164 non-null  int64  
 4   genres_x                      29164 non-null  object 
 5   homepage                      3446 non-null   object 
 6   id                            29164 non-null  int64  
 7   imdb_id                       29164 non-null  object 
 8   original_language             29164 non-null  object 
 9   original_title                29164 non-null  object 
 10  overview                      26885 non-null  object 
 11  popularity                    29164 non-null  float64
 12  poster_path                   27405 non-null  object 
 13  p

In [11]:
# L’identification des acteurs les plus présents et les périodes associées 
# Dans la table title.principals.tsv.gz on récupère toutes les lignes category == 'actor' 
# On récupère de l'autre côté la table title.basics.tsv.gz avec que les films

df_acteur = pd.read_csv('BD/title.principals.tsv.gz', compression = 'gzip', sep = '\t')
df_acteur = df_acteur[df_acteur['category'] == 'actor']
df_film = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')
df_film = df_film[df_film['titleType'] == 'movie']
df_1 = pd.merge(df_acteur,
                   df_film,
                   how = 'left',
                   on = 'tconst') 
df_1 = df_1.dropna()
df_1 = df_1['nconst'].value_counts()
df_nom = pd.read_csv('BD/name.basics.tsv.gz', compression = 'gzip', sep = '\t')
df_1 = pd.merge(df_1,
                df_nom,
                how = 'left',
                on = 'nconst') 
df_1
    

MemoryError: Unable to allocate 3.32 GiB for an array with shape (5, 89116085) and data type object

In [ ]:
df_1.groupby('nconst')['startYear'].min()
df_2 = pd.merge(df_2,
                df_2.groupby('nconst')['startYear'].min(),
                how = 'left',
                on = 'nconst') 
df_2

KeyError: 'Column not found: startYear'

In [9]:
df_kpi1 = pd.merge(df_kpi1,
                   df_kpi1.groupby('nconst')['startYear'].max(),
                   how = 'left',
                   on = 'nconst') 

KeyError: 'Column not found: startYear'

In [4]:
df_kpi1.sort_values('ordering', ascending = False)

,tconst,ordering,nconst,category,job,characters,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
751396,tt0249991,46,nm0877460,actor,\N,"[""Arriba""]",movie,The Story of Christmas,L'histoire de Noël,0,1994,\N,47,Animation,Chris Turner,\N,\N,"actor,casting_department","tt0118570,tt0491738,tt0109494,tt0116510"
751395,tt0249991,45,nm0877460,actor,\N,"[""Arriba""]",movie,The Story of Christmas,L'histoire de Noël,0,1994,\N,47,Animation,Chris Turner,\N,\N,"actor,casting_department","tt0118570,tt0491738,tt0109494,tt0116510"
751394,tt0249991,44,nm0877460,actor,\N,"[""Missy the Donkey""]",movie,The Story of Christmas,L'histoire de Noël,0,1994,\N,47,Animation,Chris Turner,\N,\N,"actor,casting_department","tt0118570,tt0491738,tt0109494,tt0116510"
751393,tt0249991,43,nm0877460,actor,\N,"[""Missy the Donkey""]",movie,The Story of Christmas,L'histoire de Noël,0,1994,\N,47,Animation,Chris Turner,\N,\N,"actor,casting_department","tt0118570,tt0491738,tt0109494,tt0116510"
751392,tt0249991,42,nm0877460,actor,\N,"[""Shepard #2""]",movie,The Story of Christmas,L'histoire de Noël,0,1994,\N,47,Animation,Chris Turner,\N,\N,"actor,casting_department","tt0118570,tt0491738,tt0109494,tt0116510"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848373,tt0305333,1,nm0413417,actor,\N,"[""Donny the Bull""]",movie,Breakout,Breakout,0,1984,\N,61,"Adventure,Family",David Jackson,1934,2005,"actor,archive_footage","tt0076987,tt0396316,tt0162815,tt0078670"
1200112,tt11244604,1,nm5673317,actor,\N,"[""Superman""]",movie,Supermen: Guardians of Hope,Supermen: Guardians of Hope,0,\N,\N,\N,"Action,Drama",Kevin Caliber,\N,\N,"actor,producer,stunts","tt4975856,tt2729716,tt17069062,tt10346834"
379919,tt0086852,1,nm0006939,actor,\N,"[""Bruno Vinclert""]",movie,L'addition,L'addition,0,1984,\N,85,"Drama,Thriller",Richard Berry,1950,\N,"actor,writer,director","tt1167638,tt0310203,tt3913206,tt0105131"
2191936,tt5219274,1,nm2355852,actor,\N,\N,movie,Isiboshwa,Isiboshwa,0,1989,\N,73,Adventure,Innocent Gumede,\N,\N,actor,"tt5212696,tt0843363,tt6313042,tt6313048"


In [5]:
df_kpi1.groupby('nconst')['startYear'].min()

nconst
nm0000001    1933
nm0000004    1978
nm0000005    1950
nm0000007    1930
nm0000008    1950
             ... 
nm9993650    2018
nm9993652    2018
nm9993656    2018
nm9993680    2019
nm9993694    2023
Name: startYear, Length: 782738, dtype: object